In [1]:
!pip install xgboost


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv("House Price India.csv")
df.head()

,id,Date,number of bedrooms,number of bathrooms,living area,lot area,number of floors,waterfront present,number of views,condition of the house,...,Built Year,Renovation Year,Postal Code,Lattitude,Longitude,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport,Price
0,6762810635,42491,4,2.50,2920,4000,1.5,0,0,5,...,1909,0,122004,52.8878,-114.470,2470,4000,2,51,1400000
1,6762810998,42491,5,2.75,2910,9480,1.5,0,0,3,...,1939,0,122004,52.8852,-114.468,2940,6600,1,53,1200000
2,6762812605,42491,4,2.50,3310,42998,2.0,0,0,3,...,2001,0,122005,52.9532,-114.321,3350,42847,3,76,838000
3,6762812919,42491,3,2.00,2710,4500,1.5,0,0,4,...,1929,0,122006,52.9047,-114.485,2060,4500,1,51,805000
4,6762813105,42491,3,2.50,2600,4750,1.0,0,0,4,...,1951,0,122007,52.9133,-114.590,2380,4750,1,67,790000


In [3]:
df.dropna(inplace=True)

In [4]:
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [7]:
X = df.drop('Price', axis=1)  
y = df['Price']               
print(X)
print(y)

               id   Date  number of bedrooms  number of bathrooms  \
0      6762810635  42491                   4                 2.50   
1      6762810998  42491                   5                 2.75   
2      6762812605  42491                   4                 2.50   
3      6762812919  42491                   3                 2.00   
4      6762813105  42491                   3                 2.50   
...           ...    ...                 ...                  ...   
14614  6762830250  42734                   2                 1.50   
14615  6762830339  42734                   3                 2.00   
14616  6762830618  42734                   2                 1.00   
14617  6762830709  42734                   4                 1.00   
14618  6762831463  42734                   3                 1.00   

       living area  lot area  number of floors  waterfront present  \
0             2920      4000               1.5                   0   
1             2910      9480   

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train, X_test, y_train, y_test )

               id   Date  number of bedrooms  number of bathrooms  \
2996   6762829702  42538                   3                 1.75   
1369   6762828931  42512                   3                 1.75   
740    6762828591  42503                   3                 2.50   
9527   6762827655  42636                   3                 2.00   
10324  6762830208  42650                   3                 1.50   
...           ...    ...                 ...                  ...   
5191   6762815533  42568                   4                 2.50   
13418  6762830172  42706                   3                 2.50   
5390   6762813778  42572                   5                 3.50   
860    6762819466  42505                   3                 2.00   
7270   6762822488  42600                   3                 2.50   

       living area  lot area  number of floors  waterfront present  \
2996          1670      6900               1.0                   0   
1369          1620      7540   

In [10]:
model_default = XGBRegressor()
model_default.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [13]:
y_pred_default = model_default.predict(X_test)
print("y_pred_default:",y_pred_default)

y_pred_default: [ 537632.3   553842.56  393968.8  ...  696166.06  444663.38 1752093.1 ]


In [14]:
mse_default = mean_squared_error(y_test, y_pred_default)
r2_default = r2_score(y_test, y_pred_default)
print("mse_default:",mse_default)
print("r2_default:",r2_default)

mse_default: 4782696448.0
r2_default: 0.9676836133003235


In [15]:
param_grid = {
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200]
}

grid_search = GridSearchCV(
    estimator=XGBRegressor(),
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None...
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1], 'max_depth': [3, 5],
                         'n_estimators': [100, 200]},
             scoring='neg_mean_squared_error', verbose=1)

In [16]:
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

best_model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)


In [17]:
print("best_model:",best_model)
print("y_pred_best:",y_pred_best)
print("mse_besr:",mse_best)
print("r2_best:",r2_best)

best_model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)
y_pred_best: [ 536922.3   552100.3   394627.94 ...  693002.4   443773.66 1874949.5 ]
mse_besr: 4320057856.0
r2_best: 0.9708096385002136


In [18]:
results = pd.DataFrame({
    'Model': ['Default XGBoost', 'Tuned XGBoost'],
    'MSE': [mse_default, mse_best],
    'R2 Score': [r2_default, r2_best]
})

print("🔍 Best Parameters:", grid_search.best_params_)
print("\n📈 Model Comparison:\n", results)


🔍 Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}

📈 Model Comparison:
              Model           MSE  R2 Score
0  Default XGBoost  4.782696e+09  0.967684
1    Tuned XGBoost  4.320058e+09  0.970810
